In [7]:
import pandas as pd
from datetime import datetime

In [9]:
today = datetime.today()
cur_year = today.year

def days_since(strin):
    try:
        date = datetime.strptime(strin, '%m/%d/%y')
        delta = today - date
        # If %y < 70, strptime defaults to 2000s. check for 1916-60s.
        # count on nothing being unchecked for 100 years. I checked; seems true
        days = int(delta.days)
        if days < 0:
            days += 36500  # 100 years
        return days
    except ValueError:
        try:
            date = datetime.strptime(strin, '%Y-%m')  # full year specified
            delta = today - date
            return delta.days
        except:
            pass


def to_bool(label):
    return bool(label)

In [24]:
csv_pddf = pd.read_csv('dashboard_012923.csv',
                       header=0,
                       names=['Bed', 'Label', 'Geo?', 'Days Since Sighted', 'Taxon'],
                       converters={'Geo?': to_bool,
                                   'Label': to_bool,
                                   'Days Since Sighted': days_since
                                   },
                       dtype={'Bed': 'category',
                              'Status': 'category',
                              }
                       )

In [25]:
csv_pddf.head()

,Bed,Label,Geo?,Days Since Sighted,Taxon
0,1e+01,False,False,6474,Quercus ilex
1,1e+01,False,False,6474,Fagus sylvatica 'Asplenifolia'
2,1e+01,False,False,6215,Hypericum calycinum
3,1e+01,False,False,6470,Quercus palustris
4,1e+01,True,False,507,Prunus 'Pink Perfection'
